In [1]:
import os
import json
import time
import subprocess
from pathlib import Path
import shlex
from datetime import datetime

import h5py
import numpy as np
from openpyxl import Workbook, load_workbook



   ###================================================================================================================================###
   
                                        ###Creating JSON File if performing auto ptyychography###

   ###================================================================================================================================###

                     #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Template selection ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
'''
   When creating the json files for ptychographic recon we need to use a template that matches the conditions 4D-STEM data was taken with 
'''

TEMPLATE_MAP = {
    "80 KeV": "/dls_sw/e02/PtyREX_templates/80KeV_template.json",
    "200 KeV": "/dls_sw/e02/PtyREX_templates/200KeV_template.json",
    "300 KeV": "/dls_sw/e02/PtyREX_templates/300KeV_template.json",
    "300 KeV (Multi-slice)": "/dls_sw/e02/PtyREX_templates/300KeV_template_ms.json",
}

def _in_notebook() -> bool:
    try:
        from IPython import get_ipython  
        ip = get_ipython()
        if ip is None:
            return False
        return "IPKernelApp" in ip.config
    except Exception:
        return False


def select_template_cli(templates: dict[str, str]) -> str:
    """
    Terminal prompt selection.
    Returns template_json_path.
    """
    keys = list(templates.keys())
    print("\nSelect PtyREX template:")
    for i, k in enumerate(keys, 1):
        print(f"  {i}) {k} -> {templates[k]}")
    while True:
        choice = input(f"Enter choice [1-{len(keys)}] (or paste full path): ").strip()
        if not choice:
            continue
        if choice.startswith("/"):
            return choice
        if choice.isdigit():
            idx = int(choice)
            if 1 <= idx <= len(keys):
                return templates[keys[idx - 1]]
        print("Invalid selection, try again.")


def ask_bool_cli(prompt: str, default: bool) -> bool:
    suffix = "Y/n" if default else "y/N"
    while True:
        s = input(f"{prompt} [{suffix}]: ").strip().lower()
        if s == "":
            return default
        if s in ("y", "yes"):
            return True
        if s in ("n", "no"):
            return False
        print("Please enter y or n.")


         #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Auto customisation of JSON template ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
'''
                          Purpose of this is to impose your experimental conditions into the JSON template.
          This generally helps initialise the probe during the ptychographic reconstruction by establishing key parameters.
                                         Parameters adjusted are shown in the table below.
'''
             # +----------------------+-----------------------------------------------+-------------------------------+ # 
             # | Parameter            | JSON path                                     | Source / Notes                | # 
             # +----------------------+-----------------------------------------------+-------------------------------+ # 
             # | Scan rotation        | process.common.scan.rotation                  | Voltage-dependent offset      | # 
             # | Camera length        | experiment.detector.position[2]               | factor * nominal CL           | # 
             # | Convergence angle    | experiment.optics.lens.alpha                  | 2 × semi-conv (aperture map)  | # 
             # | Scan shape (N)       | process.common.scan.N                         | metadata or inferred          | # 
             # | Scan step size       | process.common.scan.dR                        | metadata step_size(m)         | # 
             # | Beam energy          | process.common.source.energy                  | acceleration voltage          | # 
             # | Defocus              | experiment.optics.lens.defocus                | metadata defocus (nm → m)     | # 
             # | Data path            | experiment.data.data_path                     | input HDF5 file               | #
             # | Output directories   | base_dir / process.save_dir                   | out_dir                       | #
             # +----------------------+-----------------------------------------------+-------------------------------+ #



def meta2config(acc_v, nominal_cl_m, aperture_id, nominal_rot, factor=1.7):
    conv_angle = 0.0  
    if np.isclose(acc_v, 80e3):
        rot_angle = 238.5 - nominal_rot
        conv_map = {1: 41.65e-3, 2: 31.74e-3, 3: 24.80e-3, 4: 15.44e-3}
        conv_angle = conv_map.get(int(aperture_id), 0.0)
    elif np.isclose(acc_v, 200e3):
        rot_angle = -77.585 - nominal_rot
        conv_map = {1: 37.7e-3, 2: 28.8e-3, 3: 22.4e-3, 4: 14.0e-3, 5: 6.4e-3}
        conv_angle = conv_map.get(int(aperture_id), 0.0)
    elif np.isclose(acc_v, 300e3):
        rot_angle = -85.5 - nominal_rot
        conv_map = {1: 44.7e-3, 2: 34.1e-3, 3: 26.7e-3, 4: 16.7e-3}
        conv_angle = conv_map.get(int(aperture_id), 0.0)
    else:
        rot_angle = 0.0

    camera_length = float(factor) * float(nominal_cl_m)
    return float(rot_angle), float(camera_length), float(conv_angle)


def safe_scan_shape(h5):
    try:
        md = h5["metadata"]
        shape = md["4D_shape"][:2]
        return [int(shape[0]), int(shape[1])]
    except Exception:
        pass

    try:
        n = int(h5["data"]["frames"].shape[0])
    except Exception:
        return None

    mapping = {
        262144: (512, 512),
        261632: (512, 511),
        261121: (511, 511),
        65536:  (256, 256),
        65280:  (256, 255),
        65025:  (255, 255),
        16384:  (128, 128),
        16256:  (128, 127),
        16129:  (127, 127),
    }
    if n in mapping:
        return [mapping[n][0], mapping[n][1]]
    return None


def generate_ptyrex_json(
    meta_hdf_path,
    template_json_path,
    out_dir,
    config_name="ptycho",
    overwrite=False,
    factor=1.7,
    verbose=True,
):
    os.makedirs(out_dir, exist_ok=True)

    out_json = os.path.join(out_dir, f"{config_name}.json")
    if os.path.exists(out_json) and (not overwrite):
        if verbose:
            print(f"skip (exists): {out_json}")
        return out_json

    with open(template_json_path, "r") as f:
        cfg = json.load(f)

    with h5py.File(meta_hdf_path, "r") as h5:
        md = h5["metadata"]
        acc = float(md["ht_value(V)"][()])
        nCL = float(md["nominal_camera_length(m)"][()])
        aps = int(md["aperture_size"][()])
        rot = float(md["nominal_scan_rotation"][()])

        rot_angle, cam_len, conv = meta2config(acc, nCL, aps, rot, factor=factor)

        scanN = safe_scan_shape(h5)
        step = float(md["step_size(m)"][()])
        defocus_nm = float(md["defocus(nm)"][()])

    cfg["base_dir"] = out_dir
    cfg["process"]["save_dir"] = out_dir
    cfg["experiment"]["data"]["data_path"] = meta_hdf_path

    cfg["process"]["common"]["scan"]["rotation"] = rot_angle
    cfg["experiment"]["detector"]["position"] = [0, 0, cam_len]
    cfg["experiment"]["optics"]["lens"]["alpha"] = float(conv) * 2.0  # 2* Semi_conv angle

    if scanN is not None:
        cfg["process"]["common"]["scan"]["N"] = scanN

    cfg["process"]["common"]["source"]["energy"] = [acc]
    cfg["process"]["common"]["scan"]["dR"] = [step, step]
    cfg["experiment"]["optics"]["lens"]["defocus"] = [defocus_nm * 1e-9, defocus_nm * 1e-9]
    cfg["process"]["save_prefix"] = config_name

    with open(out_json, "w") as f:
        json.dump(cfg, f, indent=4)

    if verbose:
        print(f"wrote: {out_json}")
    return out_json
                  # ================================== End of JSON template section ================================== # 





   ###================================================================================================================================###
   
                                        ### Establishing Slurm submission via SSH (Wilson) ###  

   ###================================================================================================================================###


                                            # # # # # # # # # # # # # # # # # # # # # # #
                                            #                                           #
                                            #                 .-""""-.                  #
                                             #              .'        `.               #
                                              #            /   O    O   \             #
                                               #          :      __      :           #
                                                #         |     /  \     |          #
                                                 #        :     \__/     ;         # 
                                                  #        \            /         #
                                                   #        `.        .'         #
                                                    #         `-....-'          #
                                                     #                         #
                                                      #  ~~~~~~~~~~~~~~~~~~   #
                                                       # ~~  W I L S O N  ~~ #
                                                        # ~~~~~~~~~~~~~~~~ #



def submit_job_via_ssh(script_path, host="wilson", user=None):
    try:
        ssh_target = f"{user}@{host}" if user else host
        ssh_process = subprocess.Popen(
            ["ssh", "-tt", ssh_target],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=0,
        )

        ssh_process.stdin.write(f"sbatch {script_path}\n")
        ssh_process.stdin.write("logout\n")
        ssh_process.stdin.close()

        output_lines = ssh_process.stdout.readlines()
        ssh_process.wait(timeout=60)

        output = "".join(output_lines).strip()
        job_id = None
        success = False

        for line in output_lines:
            if "Submitted batch job" in line:
                job_id = line.strip().split()[-1]
                success = True
                break

        return success, job_id, output

    except subprocess.TimeoutExpired:
        return False, None, "SSH submission timed out"
    except Exception as e:
        return False, None, str(e)


def submit_sbatch_json_via_ssh(bash_script_path: str, json_path: str, host="wilson", user=None):
    try:
        ssh_target = f"{user}@{host}" if user else host
        ssh_process = subprocess.Popen(
            ["ssh", "-tt", ssh_target],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=0\
        )

        cmd = f"sbatch {shlex.quote(bash_script_path)} {shlex.quote(json_path)}\n"
        ssh_process.stdin.write(cmd)
        ssh_process.stdin.write("logout\n")
        ssh_process.stdin.close()

        output_lines = ssh_process.stdout.readlines()
        ssh_process.wait(timeout=60)

        output = "".join(output_lines).strip()
        job_id = None
        success = False

        for line in output_lines:
            if "Submitted batch job" in line:
                job_id = line.strip().split()[-1]
                success = True
                break

        return success, job_id, output

    except subprocess.TimeoutExpired:
        return False, None, "SSH submission timed out"
    except Exception as e:
        return False, None, str(e)


def is_slurm_job_active(job_id: str, host="wilson", user=None) -> bool:
    if not job_id:
        return False

    ssh_target = f"{user}@{host}" if user else host
    try:
        cmd = ["ssh", ssh_target, f"squeue -h -j {shlex.quote(str(job_id))} -o %T"]
        r = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        if r.returncode != 0:
            return True  
        return bool((r.stdout or "").strip())
    except Exception:
        return True  

                 # ================================== End of Slurm submission section  ================================== # 






   ###================================================================================================================================###
   
                                      ### General Functions to help avoid job submission errors ###  

   ###================================================================================================================================###


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
''' 
                                         MIB "stability" helper monitors any new mib file for changes 
                              MIB files will not be submitted until their are stationary i.e not chaning in size 
                              This should avoid sending A mib file off while it is writing to the watched folder
'''
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def is_file_stable(path: Path, min_age_sec: float = 10.0, stable_window_sec: float = 5.0) -> bool:
    try:
        st1 = path.stat()
    except FileNotFoundError:
        return False

    now = time.time()
    if (now - st1.st_mtime) < min_age_sec:
        return False

    time.sleep(stable_window_sec)

    try:
        st2 = path.stat()
    except FileNotFoundError:
        return False

    return (st1.st_size == st2.st_size) and (st2.st_size > 0)
    

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
'''
                                              Watcher helpers for locating meta file near mask.png
                                                      A minor crime but a crime nonetheless
 Similar to the mib stability helper, this avoid premature job submission of the ptychography job by making sure teh mib has fully converted.
            I use a mask.png file which is always produced last during the mib convertsion to trigger the ptycho job submission.
'''
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def find_meta_file_near(mask_folder: Path, max_up: int = 3):
    patterns = ["*.hdf", "*.h5", "*.hdf5"]

    for pat in patterns:
        files = sorted(mask_folder.glob(pat))
        if files:
            return files[0]

    for sub in mask_folder.iterdir():
        if not sub.is_dir():
            continue
        for pat in patterns:
            files = sorted(sub.glob(pat))
            if files:
                return files[0]

    cur = mask_folder
    for _ in range(max_up):
        cur = cur.parent
        for pat in patterns:
            files = sorted(cur.glob(pat))
            if files:
                return files[0]

    return None


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
'''
                                                   PTYREX SUBMISSION LOCK+SENT FLAG HELPER
  The watcher searchers for all ptycho JSON files and submits them. To avoid resubmitting those reconstructions we create a "pty_sent.txt"
                    When the json is submitted this txt file appears and then acts as a flag avoiding repeat submission
'''
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def try_acquire_pty_lock(out_dir: Path) -> bool:
    lock_path = out_dir / "pty_submitting.lock"
    try:
        fd = os.open(str(lock_path), os.O_CREAT | os.O_EXCL | os.O_WRONLY)
        with os.fdopen(fd, "w") as f:
            f.write(f"pid={os.getpid()}\n")
            f.write(f"timestamp={datetime.now().isoformat(timespec='seconds')}\n")
        return True
    except FileExistsError:
        return False


def release_pty_lock(out_dir: Path):
    lock_path = out_dir / "pty_submitting.lock"
    try:
        lock_path.unlink()
    except FileNotFoundError:
        pass


def write_pty_sent_flag(out_dir: Path, job_id: str | None = None):
    flag_path = out_dir / "pty_sent.txt"
    ts = datetime.now().isoformat(timespec="seconds")

    content = "pty_sent\n"
    if job_id:
        content += f"job_id={job_id}\n"
    content += f"timestamp={ts}\n"

    flag_path.write_text(content)
    return flag_path


def should_skip_pty_submission(out_dir: Path) -> tuple[bool, str]:
    sent_flag = out_dir / "pty_sent.txt"
    no_resubmit_flag = out_dir / "pty_noresubmit.txt"
    guard_flag = out_dir / "pty_submitted.guard"

    if sent_flag.exists():
        return True, f"already submitted (found {sent_flag.name})"
    if guard_flag.exists():
        return True, f"already submitted (found {guard_flag.name})"
    if no_resubmit_flag.exists():
        return True, f"no-resubmit marker present (found {no_resubmit_flag.name})"
    return False, ""


def canonical(p: Path) -> Path:
    try:
        return p.resolve()
    except Exception:
        return p


              # ================================== End of General Functions / helpers section  ================================== # 







   ###================================================================================================================================###
   
                                ### Bash script and txt file writer (used for SLURM submission) ###  

   ###================================================================================================================================###



# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
'''
                                                        CONVERT INFO TXT WRITER
                        This txt file is important as it defines the method in which the MIB conversion should occur.
                Once these conditions are defined they make a "convert_info.txt" file which a subsequent bash script points to.
'''
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def _reshaping_flags(reshaping: str):
    if reshaping == "Auto_reshape":
        return True, False, False, False
    elif reshaping == "Flyback":
        return False, False, True, False
    elif reshaping == "Known_shape":
        return False, False, False, True
    else:
        return False, True, False, False


def write_convert_info_from_params(
    to_convert_paths,
    info_path: Path,
    reshaping="Auto_reshape",
    Scan_X=0,
    Scan_Y=0,
    bin_sig_widget=2,
    bin_nav_widget=2,
    create_virtual_image=True,
    mask_path="",
    disk_lower_thresh=0.01,
    disk_upper_thresh=0.15,
    DPC_check=True,
    parallax_check=True,
    iBF=True,
    software_basedir="/dls_sw/e02/software/epsic_tools/epsic_tools/mib2hdfConvert/MIB_convert_widget/scripts/",
):
    auto_reshape, no_reshaping, use_fly_back, known_shape = _reshaping_flags(reshaping)

    if bin_sig_widget != 1:
        bin_sig_flag, bin_sig_factor = 1, bin_sig_widget
    else:
        bin_sig_flag, bin_sig_factor = 0, bin_sig_widget

    if bin_nav_widget != 1:
        bin_nav_flag, bin_nav_factor = 1, bin_nav_widget
    else:
        bin_nav_flag, bin_nav_factor = 0, bin_nav_widget

    if not mask_path:
        mask_path = os.path.join(software_basedir, "29042024_12bitmask.h5")

    content = (
        f"to_convert_paths = {[str(p) for p in to_convert_paths]}\n"
        f"auto_reshape = {auto_reshape}\n"
        f"no_reshaping = {no_reshaping}\n"
        f"use_fly_back = {use_fly_back}\n"
        f"known_shape = {known_shape}\n"
        f"Scan_X = {Scan_X}\n"
        f"Scan_Y = {Scan_Y}\n"
        f"iBF = {iBF}\n"
        f"bin_sig_flag = {bin_sig_flag}\n"
        f"bin_sig_factor = {bin_sig_factor}\n"
        f"bin_nav_flag = {bin_nav_flag}\n"
        f"bin_nav_factor = {bin_nav_factor}\n"
        f"reshaping = {reshaping}\n"
        f"create_virtual_image = {create_virtual_image}\n"
        f"mask_path = {mask_path}\n"
        f"disk_lower_thresh = {disk_lower_thresh}\n"
        f"disk_upper_thresh = {disk_upper_thresh}\n"
        f"DPC = {DPC_check}\n"
        f"parallax = {parallax_check}\n"
    )
    info_path.write_text(content)


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# SLURM script writer for MIB conversion
# Creates the bash script which is used in combination with the convert_info.txt for MIB conversion
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def create_cluster_submit(convert_info_path: Path, array_max_index: int, submit_path: Path,
                          scripts_dir: Path,
                          MIB_SCRIPT_PATH: str):
    content = f"""#!/usr/bin/env bash
#SBATCH --partition cs04r
#SBATCH --job-name mib_convert
#SBATCH --nodes 1
#SBATCH --tasks-per-node 1
#SBATCH --cpus-per-task 1
#SBATCH --time 05:00:00
#SBATCH --mem 192G
#SBATCH --array=0-{array_max_index}%1
#SBATCH --error={scripts_dir}/%j_error.err
#SBATCH --output={scripts_dir}/%j_output.out

set -x
cd {scripts_dir}
module load python/epsic3.10

export OMP_NUM_THREADS=${{SLURM_CPUS_PER_TASK}}
export BLOSC_NTHREADS=$((SLURM_CPUS_PER_TASK * 2))
sleep 10

python {MIB_SCRIPT_PATH} {convert_info_path} $SLURM_ARRAY_TASK_ID
"""
    submit_path.write_text(content)
    submit_path.chmod(0o755)

         # ================================== End of Bash script and txt file writer section  ================================== # 





   ###================================================================================================================================###
   
                                          ### Json file Watcher and PtyREX submission ###  

   ###================================================================================================================================###



# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
'''
                                                              WATCHER
                                            creates JSON then submit PtyREX job on wilson
'''
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def ensure_pty_out_when_mask_exists(
    processed_root: Path,
    done: set,
    template_json_path: str,
    ptyrex_bash_script_path: str,
    ssh_host: str = "wilson",
    ssh_user=None,
    factor: float = 1.7,
    overwrite_json: bool = False,
    config_name: str = "ptycho",
    verbose: bool = True,
    max_submissions_per_pass: int = 1,
):
    if not processed_root.exists():
        return []

    submitted_job_ids: list[str] = []

    for mask_png in processed_root.rglob("mask.png"):
        mask_folder = mask_png.parent

        meta_path = find_meta_file_near(mask_folder, max_up=3)
        if meta_path is None:
            if verbose:
                print(f"[watch] mask present but no meta file near {mask_folder}")
            continue

        out_dir = canonical(meta_path.parent / "pty_out" / "initial_recon")
        out_dir.mkdir(parents=True, exist_ok=True)

        if out_dir in done:
            continue

        skip, reason = should_skip_pty_submission(out_dir)
        if skip:
            if verbose:
                print(f"[watch] skip: {reason} in {out_dir}")
            done.add(out_dir)
            continue

        if not try_acquire_pty_lock(out_dir):
            if verbose:
                print(f"[watch] skip: submission already in progress (lock exists in {out_dir})")
            continue

        try:
            out_json = generate_ptyrex_json(
                meta_hdf_path=str(meta_path),
                template_json_path=template_json_path,
                out_dir=str(out_dir),
                config_name=config_name,
                overwrite=overwrite_json,
                factor=factor,
                verbose=verbose,
            )

            success, job_id, msg = submit_sbatch_json_via_ssh(
                bash_script_path=ptyrex_bash_script_path,
                json_path=str(out_json),
                host=ssh_host,
                user=ssh_user,
            )

            if success:
                if verbose:
                    print(f"[watch] PtyREX submitted for {out_json} (job {job_id})")

                write_pty_sent_flag(out_dir=out_dir, job_id=job_id)
                done.add(out_dir)

                if job_id:
                    submitted_job_ids.append(job_id)

                if len(submitted_job_ids) >= max_submissions_per_pass:
                    if verbose:
                        print(f"[watch] reached max_submissions_per_pass={max_submissions_per_pass}; stopping this poll.")
                    return submitted_job_ids
            else:
                print(f"[watch] PtyREX submission failed for {out_json}: {msg}")

        except Exception as e:
            print(f"[watch] JSON generation or submission failed for {meta_path}: {e}")

        finally:
            release_pty_lock(out_dir)

    return submitted_job_ids
       # ================================== End of Json file Watcher and PtyREX submission section  ================================== # 







   ###================================================================================================================================###
   
                                                         ### Excel logger ###  

   ###================================================================================================================================###



# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
''' 
                                                        EXCEL LOGGING HELPER
                     Not so important. I wanted to make a way of tracking the progress of a ptycho reconstruction
                  If a large number of PtyREX recons are running, it may be hard to track what data is ready to view
         This excel sheet is designed to record data from any pty_out folder that a hdf file appears in (i.e your reconstruction)
    I added a 600second delay in the logger, as sometimes it takes a few iteration before any observable data is indicated in the hdf
                             I need to update this section as a lot of information recording is verbose 
'''
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def _get_nested(d, keys, default=None):
    cur = d
    for k in keys:
        if not isinstance(cur, dict) or k not in cur:
            return default
        cur = cur[k]
    return cur


def find_json_near(path: Path, prefer_name: str = "ptycho.json"):
    p1 = path.parent / prefer_name
    if p1.exists():
        return p1

    js = sorted(path.parent.glob("*.json"))
    if js:
        return js[0]

    p2 = path.parent.parent / prefer_name
    if p2.exists():
        return p2

    js2 = sorted(path.parent.parent.glob("*.json"))
    if js2:
        return js2[0]

    return None


def extract_recon_params_from_json(cfg: dict):
    cam_len_m = None
    pos = _get_nested(cfg, ["experiment", "detector", "position"], default=None)
    if isinstance(pos, (list, tuple)) and len(pos) >= 3:
        cam_len_m = pos[2]

    defocus = (
        _get_nested(cfg, ["experiment", "optics", "lens", "defocus"], default=None)
        or _get_nested(cfg, ["experiment", "optics", "lens", "defocus_m"], default=None)
        or _get_nested(cfg, ["process", "common", "optics", "lens", "defocus"], default=None)
    )
    defocus_m = None
    if isinstance(defocus, (list, tuple)) and len(defocus) > 0:
        defocus_m = defocus[0]
    elif isinstance(defocus, (int, float)):
        defocus_m = defocus

    lens_alpha_rad = (
        _get_nested(cfg, ["experiment", "optics", "lens", "alpha"], default=None)
        or _get_nested(cfg, ["process", "common", "optics", "lens", "alpha"], default=None)
    )

    probe_alpha = (
        _get_nested(cfg, ["process", "common", "probe", "alpha"], default=None)
        or _get_nested(cfg, ["process", "common", "probes", "alpha"], default=None)
        or _get_nested(cfg, ["process", "reconstruction", "probe", "alpha"], default=None)
    )
    object_alpha = (
        _get_nested(cfg, ["process", "common", "object", "alpha"], default=None)
        or _get_nested(cfg, ["process", "reconstruction", "object", "alpha"], default=None)
    )

    slice_thickness_m = (
        _get_nested(cfg, ["process", "common", "object", "slice_thickness"], default=None)
        or _get_nested(cfg, ["process", "common", "object", "thickness"], default=None)
        or _get_nested(cfg, ["process", "common", "object", "dz"], default=None)
    )

    slice_number = (
        _get_nested(cfg, ["process", "common", "object", "slice_number"], default=None)
        or _get_nested(cfg, ["process", "common", "object", "nslices"], default=None)
        or _get_nested(cfg, ["process", "common", "object", "n_slices"], default=None)
    )
    if slice_number is None:
        slices = _get_nested(cfg, ["process", "common", "object", "slices"], default=None)
        if isinstance(slices, list):
            slice_number = len(slices)

    energy = _get_nested(cfg, ["process", "common", "source", "energy"], default=None)
    acc_v = None
    if isinstance(energy, (list, tuple)) and len(energy) > 0:
        acc_v = energy[0]
    elif isinstance(energy, (int, float)):
        acc_v = energy

    return {
        "acc_v": acc_v,
        "camera_length_m": cam_len_m,
        "defocus_m": defocus_m,
        "lens_alpha_rad": lens_alpha_rad,
        "probe_alpha": probe_alpha,
        "object_alpha": object_alpha,
        "slice_thickness_m": slice_thickness_m,
        "slice_number": slice_number,
    }


def ensure_excel_log(excel_path: Path, sheet_name: str = "ready_to_view"):
    headers = [
        "timestamp",
        "message",
        "recon_hdf_path",
        "json_path",
        "acc_v",
        "camera_length_m",
        "defocus_m",
        "lens_alpha_rad",
        "probe_alpha",
        "object_alpha",
        "slice_thickness_m",
        "slice_number",
    ]

    excel_path.parent.mkdir(parents=True, exist_ok=True)

    if excel_path.exists():
        wb = load_workbook(excel_path)
        if sheet_name in wb.sheetnames:
            ws = wb[sheet_name]
            if ws.max_row == 0:
                ws.append(headers)
        else:
            ws = wb.create_sheet(sheet_name)
            ws.append(headers)
        wb.save(excel_path)
        return

    wb = Workbook()
    ws = wb.active
    ws.title = sheet_name
    ws.append(headers)
    wb.save(excel_path)


def append_ready_row_to_excel(
    excel_path: Path,
    message: str,
    recon_hdf_path: Path,
    json_path: Path | None,
    params: dict,
    sheet_name: str = "ready_to_view",
):
    ensure_excel_log(excel_path, sheet_name=sheet_name)

    wb = load_workbook(excel_path)
    ws = wb[sheet_name]

    row = [
        datetime.now().isoformat(timespec="seconds"),
        message,
        str(recon_hdf_path),
        str(json_path) if json_path else "",
        params.get("acc_v"),
        params.get("camera_length_m"),
        params.get("defocus_m"),
        params.get("lens_alpha_rad"),
        params.get("probe_alpha"),
        params.get("object_alpha"),
        params.get("slice_thickness_m"),
        params.get("slice_number"),
    ]
    ws.append(row)
    wb.save(excel_path)


def watch_ptyrex_hdf_ready(
    processed_root: Path,
    seen_hdfs: set,
    excel_log_path: Path,
    delay_sec: float = 600,
    verbose: bool = True,
    prefer_json_name: str = "ptycho.json",
):
    if not processed_root.exists():
        return 0

    triggered = 0

    for pty_out_dir in processed_root.rglob("pty_out"):
        if not pty_out_dir.is_dir():
            continue

        for hdf_path in pty_out_dir.rglob("*.hdf"):
            hdf_path = canonical(hdf_path)
            if hdf_path in seen_hdfs:
                continue

            seen_hdfs.add(hdf_path)
            folder_id = hdf_path.parent.name

            if verbose:
                print(f"[watch] detected recon HDF (pty_out): {hdf_path} (delaying {delay_sec}s)")

            time.sleep(delay_sec)

            message = f"ptycho {folder_id} ready to view"
            json_path = find_json_near(hdf_path, prefer_name=prefer_json_name)

            params = {
                "acc_v": None,
                "camera_length_m": None,
                "defocus_m": None,
                "lens_alpha_rad": None,
                "probe_alpha": None,
                "object_alpha": None,
                "slice_thickness_m": None,
                "slice_number": None,
            }

            if json_path and json_path.exists():
                try:
                    with open(json_path, "r") as f:
                        cfg = json.load(f)
                    params = extract_recon_params_from_json(cfg)
                except Exception as e:
                    if verbose:
                        print(f"[watch] could not parse json {json_path}: {e}")
            else:
                if verbose:
                    print(f"[watch] no json found near recon: {hdf_path}")

            try:
                append_ready_row_to_excel(
                    excel_path=excel_log_path,
                    message=message,
                    recon_hdf_path=hdf_path,
                    json_path=json_path,
                    params=params,
                )
                if verbose:
                    print(f"[watch] logged to Excel: {excel_log_path}")
            except Exception as e:
                print(f"[watch] FAILED to write Excel log for {hdf_path}: {e}")

            triggered += 1

    return triggered
    
                   # ================================== End of Excel logger section  ================================== # 






   ###================================================================================================================================###
   
                                                           ### MAIN BODY ###
                                           ### Loop, watcher, and submission execution ###

   ###================================================================================================================================###


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

'''
                                                        MAIN WATCH LOOP RUNNER
    Continuously monitors the experiment and processing directories and automatically submits SLURM jobs for MIB conversion and PtyREX
                                                             reconstruction.

    New MIB files are only processed once they are stable, job submission is rate-limited to avoid overloading the cluster, and completed
                                                 reconstructions are logged for viewing.
'''
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def run_watch_loop(
    year: str,
    visit_id: str,
    experiment_name: str,
    template_json_path: str,
    iBF: bool,
    create_virtual_image: bool,
    DPC_check: bool,
    parallax_check: bool,
):
    MIB_SCRIPT_PATH = "/dls_sw/e02/software/epsic_tools/epsic_tools/mib2hdfConvert/MIB_convert_widget/scripts/MIB_convert_submit.py"
    PTYREX_BASH_SCRIPT = "/dls/science/groups/e02/Joshua/Wilson_BashScript/Auto_ptycho_bash.sh"

    SSH_HOST = "wilson"
    SSH_USER = None  

    ROOT_DIR = Path("/dls/e02/data") / year / visit_id / "Merlin" / experiment_name
    scripts_dir = ROOT_DIR / "scripts"
    scripts_dir.mkdir(parents=True, exist_ok=True)

    PROCESSED_ROOT = Path(str(ROOT_DIR).replace("/Merlin/", "/processing/Merlin/"))

    excel_log_path = PROCESSED_ROOT / "ptyrex_ready_to_view_log.xlsx"
    ensure_excel_log(excel_log_path)

    print(f"\nExperiment directory:\n  {ROOT_DIR}")
    print(f"Scripts directory:\n  {scripts_dir}")
    print(f"Processed directory (watched):\n  {PROCESSED_ROOT}")
    print(f"Excel log:\n  {excel_log_path}")
    print(f"PtyREX template:\n  {template_json_path}\n")

    print("Conversion options:")
    print(f"  iBF = {iBF}")
    print(f"  create_virtual_image = {create_virtual_image}")
    print(f"  DPC = {DPC_check}")
    print(f"  parallax = {parallax_check}\n")

    ptyout_submitted_for = set()
    seen_recon_hdfs = set()

    MAX_ACTIVE_MIB_JOBS = 4
    MAX_ACTIVE_PTYREX_JOBS = 6

    active_mib_job_ids: set[str] = set()
    active_ptyrex_job_ids: set[str] = set()

    seen_mibs: set[Path] = set()
    pending_mibs: list[Path] = []
    mib_candidates: dict[Path, float] = {}

    MIB_BATCH_SIZE = 4
    MIB_MIN_AGE_SEC = 10.0
    MIB_STABLE_WINDOW_SEC = 5.0

    print("Watching for new *_data.mib files (queued, max 4 active conversion jobs)...")
    print("  - will only enqueue MIBs once they are stable (size not changing + old enough)")
    print("Watching for mask.png + meta file to create JSON and submit PtyREX (queued, max 4 active PtyREX jobs)...")
    print("Watching for recon *.hdf (ONLY under pty_out) to log 'ready to view' into Excel...\n")
    POLL_INTERVAL = 1

    while True:
        try:
            # 1) Refresh active job lists
            for jid in list(active_mib_job_ids):
                if not is_slurm_job_active(jid, host=SSH_HOST, user=SSH_USER):
                    active_mib_job_ids.discard(jid)

            for jid in list(active_ptyrex_job_ids):
                if not is_slurm_job_active(jid, host=SSH_HOST, user=SSH_USER):
                    active_ptyrex_job_ids.discard(jid)

            # 2) PtyREX submissions
            available_ptyrex = MAX_ACTIVE_PTYREX_JOBS - len(active_ptyrex_job_ids)
            if available_ptyrex > 0:
                new_job_ids = ensure_pty_out_when_mask_exists(
                    processed_root=PROCESSED_ROOT,
                    done=ptyout_submitted_for,
                    template_json_path=template_json_path,
                    ptyrex_bash_script_path=PTYREX_BASH_SCRIPT,
                    ssh_host=SSH_HOST,
                    ssh_user=SSH_USER,
                    factor=1.7,
                    overwrite_json=False,
                    config_name="ptycho",
                    verbose=True,
                    max_submissions_per_pass=min(available_ptyrex, 4),
                )
                for jid in new_job_ids:
                    if jid:
                        active_ptyrex_job_ids.add(jid)

                if new_job_ids:
                    print(f"[watch] PtyREX submitted {len(new_job_ids)} job(s). "
                          f"active {len(active_ptyrex_job_ids)}/{MAX_ACTIVE_PTYREX_JOBS}")

            # 3) Recon logging
            watch_ptyrex_hdf_ready(
                processed_root=PROCESSED_ROOT,
                seen_hdfs=seen_recon_hdfs,
                excel_log_path=excel_log_path,
                delay_sec=10,
                verbose=True,
                prefer_json_name="ptycho.json",
            )

            # 4) Discover new MIBs
            for mib in sorted(ROOT_DIR.rglob("*_data.mib")):
                mib = canonical(mib)
                if mib in seen_mibs:
                    continue
                mib_candidates.setdefault(mib, time.time())

            # Promote candidates only when stable
            promote_list = []
            for mib in list(mib_candidates.keys()):
                if not mib.exists():
                    mib_candidates.pop(mib, None)
                    continue

                if is_file_stable(mib, min_age_sec=MIB_MIN_AGE_SEC, stable_window_sec=MIB_STABLE_WINDOW_SEC):
                    promote_list.append(mib)
                    mib_candidates.pop(mib, None)

            for mib in promote_list:
                seen_mibs.add(mib)
                pending_mibs.append(mib)
                print(f"[watch] MIB is stable; queued for conversion: {mib}")

            # 5) Submit MIB conversion jobs up to available slots
            available_mib = MAX_ACTIVE_MIB_JOBS - len(active_mib_job_ids)
            if available_mib > 0 and pending_mibs:
                for _ in range(available_mib):
                    if not pending_mibs:
                        break

                    batch = pending_mibs[:MIB_BATCH_SIZE]
                    pending_mibs = pending_mibs[MIB_BATCH_SIZE:]

                    ts = time.time_ns()  # avoid collisions
                    temp_convert_info = scripts_dir / f"convert_info_{ts}.txt"

                    write_convert_info_from_params(
                        to_convert_paths=batch,
                        info_path=temp_convert_info,
                        reshaping="Auto_reshape",
                        Scan_X=0,
                        Scan_Y=0,
                        bin_sig_widget=2,
                        bin_nav_widget=2,
                        create_virtual_image=create_virtual_image,  
                        mask_path="",
                        disk_lower_thresh=0.01,
                        disk_upper_thresh=0.15,
                        DPC_check=DPC_check,                        
                        parallax_check=parallax_check,              
                        iBF=iBF,                                    
                    )

                    temp_cluster_submit = scripts_dir / f"cluster_submit_{ts}.sh"
                    create_cluster_submit(
                        convert_info_path=temp_convert_info,
                        array_max_index=len(batch) - 1,
                        submit_path=temp_cluster_submit,
                        scripts_dir=scripts_dir,
                        MIB_SCRIPT_PATH=MIB_SCRIPT_PATH,
                    )

                    print(f"[watch] submitting MIB conversion batch: {len(batch)} file(s). "
                          f"active {len(active_mib_job_ids)}/{MAX_ACTIVE_MIB_JOBS} "
                          f"(queue remaining: {len(pending_mibs)})")

                    success, job_id, message = submit_job_via_ssh(
                        str(temp_cluster_submit), SSH_HOST, SSH_USER
                    )
                    if success and job_id:
                        active_mib_job_ids.add(job_id)
                        print(f"[watch] MIB conversion submitted: job {job_id}\n")
                    else:
                        print(f"[watch] MIB conversion submission failed: {message}\n")
                        pending_mibs = batch + pending_mibs
                        break

            time.sleep(POLL_INTERVAL)

        except KeyboardInterrupt:
            print("Stopping watch loop.")
            break


# =========================================================
# NOTEBOOK UI:
#   1) tickboxes (images)
#   2) template selection
#   3) session details
#   4) start watchers
# =========================================================
def start_notebook_ui():
    try:
        from ipywidgets import Checkbox, VBox, Button, HTML, Dropdown, Text
        from IPython.display import display
    except Exception as e:
        print(f"ipywidgets not available ({e}); falling back to CLI prompts.")
        return start_cli_flow()

    # 1) "Select images" tickboxes
    cb_iBF = Checkbox(value=True, description="iBF")
    cb_virtual = Checkbox(value=True, description="Create virtual image")
    cb_DPC = Checkbox(value=True, description="DPC")
    cb_parallax = Checkbox(value=True, description="Parallax")

    # 2) Template selection
    template_dd = Dropdown(
        options=list(TEMPLATE_MAP.keys()),
        value="300 KeV" if "300 KeV" in TEMPLATE_MAP else list(TEMPLATE_MAP.keys())[0],
        description="Template:",
    )

    # 3) Session details
    year_txt = Text(value="", description="Year:", placeholder="e.g. 2026")
    visit_txt = Text(value="", description="Visit:", placeholder="e.g. mg12345-1")
    exp_txt = Text(value="", description="Experiment:", placeholder="experiment folder name")

    # 4) Start
    run_button = Button(description="Start auto processing", button_style="success")
    status = HTML(value="<b>Step 1:</b> Select tick boxes → template → session details, then Start.")

    ui = VBox([
        status,
        HTML(value="<hr><b>1) Select images/options</b>"),
        cb_iBF, cb_virtual, cb_DPC, cb_parallax,
        HTML(value="<hr><b>2) Select PtyREX template</b>"),
        template_dd,
        HTML(value="<hr><b>3) Session details</b>"),
        year_txt, visit_txt, exp_txt,
        HTML(value="<hr>"),
        run_button,
    ])
    display(ui)

    def _on_run_clicked(_):
        year = year_txt.value.strip()
        visit = visit_txt.value.strip()
        exp = exp_txt.value.strip()
        template_json_path = TEMPLATE_MAP.get(template_dd.value, "")

        if not year or not visit or not exp:
            status.value = "<span style='color:red'><b>Please fill Year, Visit, and Experiment.</b></span>"
            return

        if not template_json_path:
            status.value = "<span style='color:red'><b>Template selection is invalid.</b></span>"
            return

        status.value = (
            "<b>Starting watchers with:</b><br>"
            f"iBF={cb_iBF.value}, create_virtual_image={cb_virtual.value}, "
            f"DPC={cb_DPC.value}, parallax={cb_parallax.value}<br>"
            f"template={template_dd.value}<br>"
            f"year={year}, visit={visit}, experiment={exp}"
        )

        run_watch_loop(
            year=year,
            visit_id=visit,
            experiment_name=exp,
            template_json_path=template_json_path,
            iBF=cb_iBF.value,
            create_virtual_image=cb_virtual.value,
            DPC_check=cb_DPC.value,
            parallax_check=cb_parallax.value,
        )

    run_button.on_click(_on_run_clicked)


# =========================================================
# CLI flow:
#   1) tickboxes (y/n)
#   2) template selection
#   3) session details
#   4) start watchers
# =========================================================
def start_cli_flow():
    print("\nStep 1) Select images/options")
    iBF = ask_bool_cli("Enable iBF?", default=True)
    create_virtual_image = ask_bool_cli("Create virtual image?", default=True)
    DPC_check = ask_bool_cli("Enable DPC?", default=True)
    parallax_check = ask_bool_cli("Enable parallax?", default=True)

    print("\nStep 2) Select PtyREX template")
    template_json_path = select_template_cli(TEMPLATE_MAP)

    print("\nStep 3) Session details")
    year = input("Enter year (yyyy): ").strip()
    visit_id = input("Enter visit ID (e.g. mg12345-1): ").strip()
    experiment_name = input("Enter experiment folder name: ").strip()

    print("\nStep 4) Starting watchers...\n")
    run_watch_loop(
        year=year,
        visit_id=visit_id,
        experiment_name=experiment_name,
        template_json_path=template_json_path,
        iBF=iBF,
        create_virtual_image=create_virtual_image,
        DPC_check=DPC_check,
        parallax_check=parallax_check,
    )


if __name__ == "__main__":
    if _in_notebook():
        start_notebook_ui()
    else:
        start_cli_flow()
